In [2]:
from azureml.core import Workspace

# creating workspace

In [3]:
ws= Workspace.create(name='Azureml-SDK-brillio',
                     subscription_id="a4df42c9-c1d2-44c4-b2c2-05edb4b27aad",
                     resource_group="RG-MSFT-ML-POC-01",
                     create_resource_group=True,
                     location="eastus")


Deploying KeyVault with name azuremlskeyvaulte1cd3676.
Deploying AppInsights with name azuremlsinsightsd606719b.
Deployed AppInsights with name azuremlsinsightsd606719b. Took 8.86 seconds.
Deploying StorageAccount with name azuremlsstorageb9cba2ee5.
Deployed KeyVault with name azuremlskeyvaulte1cd3676. Took 24.89 seconds.
Deployed StorageAccount with name azuremlsstorageb9cba2ee5. Took 32.58 seconds.
Deploying Workspace with name Azureml-SDK-brillio.
Deployed Workspace with name Azureml-SDK-brillio. Took 26.5 seconds.


In [3]:
ws=Workspace.from_config("./config")                 

In [10]:
ws.name

'Azureml-SDK-brillio'

# creating a Datastore

In [4]:
from azureml.core import Datastore
az_store = Datastore.register_azure_blob_container(workspace=ws,
                                                    datastore_name="azure_sdk_blob_logging",
                                                    account_name="sadlsmsftmlpoc01",
                                                    container_name="titanicdataset",
                                                    account_key="BHMyM/JHMpiMbVz4co6fUT+2VPlZTgMcLmPt1KEb2fM1bQ4LJpNpz7vb9npZvphEjshGoZWzrRU2+AStllFgcA==")

In [6]:
az_store= Datastore.get(ws,"azure_sdk_blob_logging")

In [7]:
az_store.name

'azure_sdk_blob_logging'

# creating a Dataset

In [10]:
from azureml.core import Dataset
csv_path= [(az_store,"titanic.csv")]
dataset= Dataset.Tabular.from_delimited_files(path=csv_path)
#register the dataset
dataset = dataset.register(workspace=ws,
                            name="Titanic_vikas_logging",
                            create_new_version=True)

In [16]:
ws_list=Workspace.list(subscription_id="a4df42c9-c1d2-44c4-b2c2-05edb4b27aad")
ws_list=list(ws_list)
az_default_store= ws.get_default_datastore()
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
ds_list=list(ws.datasets.keys())
for items in ds_list:
    print(items)

Titanic_vikas


In [17]:
ws=Workspace.from_config("./config")
az_store= Datastore.get(ws,"azure_sdk_blob01")
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
az_default_store= ws.get_default_datastore()

In [19]:
df=az_dataset.to_pandas_dataframe()
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [20]:
df=df[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked","Survived"]]

In [21]:
az_ds_from_df = Dataset.Tabular.register_pandas_dataframe(dataframe=df,
                                                          target=az_store,
                                                          name="Titanic_selected_columns")

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/f5371f5c-4192-4ec6-ba11-5639e0a1a657/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [ ]:
files_list=["./data/test.csv","./data/test1.csv"]
files_list= az_store.upload_files(files=files_list,
                                   target_path="folder_name/",
                                relative_root="./data/",
                                overwrite=True)

In [ ]:
az_store.upload(src_dir="./data",
                target_path="folder_name/data",
                overwrite=True)

# Creating Experiment

In [22]:
from azureml.core import Experiment
experiment=Experiment(workspace=ws,name="azureml_titanic_vikas")

In [25]:
null_df=df.isnull().sum()
null_df["Pclass"]

0

# Running experiment by logging simple metrics

In [26]:
new_run= experiment.start_logging()
total_observations=len(df)
df_null=df.isnull().sum()
new_run.log("Total observations: ",total_observations)
for column in df.columns:
    new_run.log(column,df_null[column])
new_run.complete()

# Creating Environment and attach to configuration

In [28]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

In [29]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

{
    "assetId": "azureml://locations/eastus/workspaces/e55a79e0-02bd-4a4c-8d23-7008f3ae598f/environments/MyEnvironment/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221010.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "MyEnvironment",
    "pyt

In [30]:
from azureml.core import ScriptRunConfig 

In [31]:
from azureml.core import Experiment
experiment=Experiment(workspace=ws,name="azureml_titanic_vikas")

# Creating Compute cluster in azure

In [40]:
from azureml.core.compute import AmlCompute 
cluster_name= "mspoctitanic1"
if cluster_name not in ws.compute_targets:
    compute_config= AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                        max_nodes=2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
    print("new cluster created successfully: ",cluster)
else:
    cluster= ws.compute_targets[cluster_name]
    print(cluster," compute cluster found.. using it....")

Provisioning operation finished, operation "Succeeded"
new cluster created successfully:  AmlCompute(workspace=Workspace.create(name='Azureml-SDK-brillio', subscription_id='a4df42c9-c1d2-44c4-b2c2-05edb4b27aad', resource_group='RG-MSFT-ML-POC-01'), name=mspoctitanic1, id=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.MachineLearningServices/workspaces/Azureml-SDK-brillio/computes/mspoctitanic1, type=AmlCompute, provisioning_state=Succeeded, location=eastus, tags={})


In [41]:
from azureml.core.compute import ComputeTarget
compute_cluster= ComputeTarget.create(ws, cluster_name, compute_config)
compute_cluster.wait_for_completion()

Provisioning operation finished, operation "Succeeded"


In [42]:
from azureml.core.runconfig import RunConfiguration 
run_config = RunConfiguration()
run_config.target= compute_cluster
run_config.environment = myenv 

# Running Experiment in azure

In [43]:
script_config = ScriptRunConfig(source_directory="Training_python_scripts/",
                                script="Singlestep_Training.py",
                                environment=myenv,
                                run_config=run_config)
new_script=experiment.submit(script_config)
new_script.wait_for_completion(True)

RunId: azureml_titanic_vikas_1667488076_f79db131
Web View: https://ml.azure.com/runs/azureml_titanic_vikas_1667488076_f79db131?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourcegroups/RG-MSFT-ML-POC-01/workspaces/Azureml-SDK-brillio&tid=3882b70d-a91e-468c-9928-820358bfbd73


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}